<a href="https://colab.research.google.com/github/apatre/apatre-beam_exercises/blob/main/BeamCollege2024_Intermediate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pip --upgrade --quiet
!pip install apache_beam[gcp,dataframe] transformers==4.37.0 accelerate pandas huggingface-hub==0.24.0 --quiet
!pip install https://h2o-release.s3.amazonaws.com/datatable/dev/datatable-1.1.0a2242/datatable-1.1.0a2242-cp310-cp310-manylinux_2_17_x86_64.whl#sha256=ff0374cced029857c8d73d79ace488d24b5b55f46a8132f5eb597b2cc525d1af --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.0/419.0 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# You need to authenticate to read the input data from GCS.
from google.colab import auth
auth.authenticate_user()

Download the files needed to preprocess data and perform classification with our XGBoost model.

These files are also available on Google Drive:
https://drive.google.com/drive/folders/14qNxb7vx62kRO4woARQMHTJ2KHPgDMzS?usp=sharing

In [ ]:
!gsutil cp gs://beam_college_intermediate/svd.pk .
!gsutil cp gs://beam_college_intermediate/vectorizer.pk .
!gsutil cp gs://beam_college_intermediate/xgb_call_classifier.json .

Copying gs://beam_college_intermediate/svd.pk...
- [1 files][312.5 KiB/312.5 KiB]                                                
Operation completed over 1 objects/312.5 KiB.                                    
Copying gs://beam_college_intermediate/vectorizer.pk...
/ [1 files][  4.6 KiB/  4.6 KiB]                                                
Operation completed over 1 objects/4.6 KiB.                                      
Copying gs://beam_college_intermediate/xgb_call_classifier.json...
\ [1 files][ 10.3 MiB/ 10.3 MiB]                                                
Operation completed over 1 objects/10.3 MiB.                                     


In [ ]:
import apache_beam as beam
from apache_beam.ml.inference import utils
from apache_beam.ml.inference.base import PredictionResult
from apache_beam.ml.inference.base import RunInference
from apache_beam.ml.inference.huggingface_inference import HuggingFacePipelineModelHandler
from apache_beam.ml.inference.xgboost_inference import XGBoostModelHandlerNumpy
from apache_beam.ml.inference.xgboost_inference import XGBoostModelHandler
from apache_beam.ml.inference.base import KeyModelMapping, KeyedModelHandler
from IPython.display import Audio, display
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from transformers import VitsModel, AutoTokenizer
from typing import Dict
from typing import Iterable
from typing import Tuple

import numpy as np
import pandas as pd
import pickle
import xgboost as xgb

# This is needed for the XGBoost classifier as part of vectorization.
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Data is from https://github.com/cricketclub/gridspace-stanford-harper-valley
The dataset primarily consists of simulated contact center calls to Harper Valley Bank in the Gridspace Mixer platform, and is meant for training machine learning models.

In [ ]:
# Speech to text model handler
# From https://huggingface.co/openai/whisper-small
pipeline_handler_whisper = HuggingFacePipelineModelHandler(
        task="automatic-speech-recognition",
        model="openai/whisper-small",
        min_batch_size=2,
        load_pipeline_args={'chunk_length_s':30, 'device': 'cuda:0'},
        large_model=True)

def post_process_whisper(result: PredictionResult):
  result = result.inference[0]['text']
  return (result, result)


In [ ]:
# Custom trained XGBoost Classifier to get call type from text transcript.
# We need to use the vectorizer and lsa (dimensionality reducer) that were fit
# on the training data.
# Loosely based on https://www.kaggle.com/code/diveki/classification-with-nlp-xgboost-and-pipelines
# and https://ai.plainenglish.io/sentiment-classification-using-xgboost-7abdaf4771f9
with open('vectorizer.pk', 'rb') as f:
    vec = pickle.load(f)

with open('svd.pk', 'rb') as f:
    svd = pickle.load(f)

# model_state is a filepath to a model created by model.save_model([filename])
xgboost_handler = XGBoostModelHandlerNumpy(model_class=xgb.XGBClassifier,
                                          model_state="xgb_call_classifier.json")
keyed_xgboost_handler = KeyedModelHandler(xgboost_handler)

# We must preprocess the data using the same, already fitted, processors as used
# in training.
def pre_process_xgboost(element: Tuple[str, str]):
  call_text = element[1]
  call_vec = vec.transform([call_text])
  call_svd = svd.transform(call_vec)
  return (element[0], call_svd)

def translate_labels(element: Tuple[str, PredictionResult]):
  """
    Maps output values (0 to 7) of the XGBoost call classification
    model to the types of calls.
    Args:
        inference_results: Array containing the outputs of the XGBoost classification model
  """
  original_text = element[0]
  inference_results = element[1]

  call_type_as_array = np.vectorize(['check balance', 'get branch hours', 'order checks', 'pay bill',
                    'replace card', 'reset password', 'schedule appointment',
                    'transfer money'].__getitem__)(inference_results.inference)
  call_type = call_type_as_array.item()
  return (call_type, original_text)

In [ ]:
# Models for the keyed model inference. Each model will serve a different key,
# in this case a class of help call. Models were chosen at random, based on being small
# enough to fit in colab. All from https://huggingface.co/models?pipeline_tag=text-generation
pipeline_args = {'framework': 'pt', 'max_length': 90, 'pad_token_id':50256, 'num_return_sequences':1, 'device': 'cuda:0'}
balance_mh = HuggingFacePipelineModelHandler("text-generation", model="distilgpt2",
                                       load_pipeline_args=pipeline_args)
checks_mh = HuggingFacePipelineModelHandler("text-generation", model="fxmarty/tiny-llama-fast-tokenizer",
                                       load_pipeline_args=pipeline_args)
pay_mh = HuggingFacePipelineModelHandler("text-generation", model="roneneldan/TinyStories-8M",
                                       load_pipeline_args=pipeline_args)
password_mh = HuggingFacePipelineModelHandler("text-generation", model="fxmarty/really-tiny-falcon-testing",
                                       load_pipeline_args=pipeline_args)
transfer_mh = HuggingFacePipelineModelHandler("text-generation", model="roneneldan/TinyStories-8M",
                                       load_pipeline_args=pipeline_args)

# Mapping keys to models is very easy.
per_key_mhs = [
  KeyModelMapping(['check balance', 'get branch hours'], balance_mh),
  KeyModelMapping(['order checks'], checks_mh),
  KeyModelMapping(['pay bill', 'replace card'], pay_mh),
  KeyModelMapping(['reset password'], password_mh),
  KeyModelMapping(['schedule appointment', 'transfer money'], transfer_mh),
]

mh = KeyedModelHandler(per_key_mhs)

# We process the results to remove the inital text from the response that we will
# pass on the text to speech model.
class ExtractResults(beam.DoFn):
  """
  Extract the relevant data from the PredictionResult object.
  """
  def process(self, element: Tuple[str, PredictionResult]):
    call_type = element[0]
    result = element[1]
    text_from_whisper = result.example
    full_reply = result.inference[0][0]['generated_text']
    llm_response_only = full_reply.replace(text_from_whisper, '')
    # We log the call type and response to evaluate the different models used.
    print('type:', call_type, '  text:', llm_response_only)
    yield (llm_response_only)

In [ ]:
# text_to_speech_tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-eng")
# Text to speech models and processing from https://huggingface.co/facebook/mms-tts-eng
model_handler_text_to_speech = HuggingFacePipelineModelHandler(
    task="text-to-speech",
    model="facebook/mms-tts-eng",
    load_pipeline_args={'framework': 'pt', 'device': 'cuda:0'}
)

# We play the audio in colab. We could also save to a file as a .wav
def post_process_text_to_speech(result: PredictionResult):
  display(Audio(result.inference[0]['audio'],
                rate=result.inference[0]['sampling_rate'],
                autoplay=False))
  return result.inference[0]

In [ ]:
# Pipeline code that ties everything together.
def main():
  """Runs a Beam pipeline that calls an ML model using RunInference."""
  pipeline = beam.Pipeline()

  with pipeline as p:
        (
        p
        | "GetInputFilenames" >> beam.io.fileio.MatchFiles('gs://beam_college_intermediate/caller/*')
        | "ReadInputData" >> beam.io.fileio.ReadMatches()
        | "WhisperSpeechToText" >> RunInference(
            pipeline_handler_whisper
            .with_preprocess_fn(
                lambda x: x.read(mime_type='audio/wav')
            )
            .with_postprocess_fn(
                lambda x: post_process_whisper(x)
            )
            )
        | "XGBoostClassifier" >> RunInference(
            keyed_xgboost_handler
            .with_preprocess_fn(
                lambda x: pre_process_xgboost(x)
            )
            .with_postprocess_fn(
                lambda x: translate_labels(x)
            ))
        | "KeyedModelHandler" >> RunInference(mh)
        | "ExtractResults" >> beam.ParDo(ExtractResults())
        | "TextToSpeech" >> RunInference(
            model_handler_text_to_speech
            .with_postprocess_fn(
                lambda x: post_process_text_to_speech(x)
            ))
        )
  # Run the pipeline.
  pipeline.run()

if __name__ == '__main__':
    main()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/145M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1136: UserWarning: Input length of input_ids is 102, but `max_length` is set to 90. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


type: get branch hours   text: 
Advertisements


ERROR:apache_beam.runners.common:Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding) [while running 'TextToSpeech/BeamML_RunInference']
Traceback (most recent call last):
  File "apache_beam/runners/common.py", line 1435, in apache_beam.runners.common.DoFnRunner.process
  File "apache_beam/runners/common.py", line 861, in apache_beam.runners.common.PerWindowInvoker.invoke_process
  File "apache_beam/runners/common.py", line 999, in apache_beam.runners.common.PerWindowInvoker._invoke_process_per_window
  File "/usr/local/lib/python3.10/dist-packages/apache_beam/ml/inference/base.py", line 1605, in process
    return self._run_inference(batch, inference_args)
  File "/usr/local/lib/python3.10/dist-packages/apache_beam/ml/inference/base.py", line 1578, in _run_inference
    raise e
  File "/usr/local/lib/python3.10/dist-packages/apache_beam/ml/inference/base.py", lin

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding) [while running 'TextToSpeech/BeamML_RunInference']

*This is the same as the original notebook with a couple of small updates to account for changes to dependencies, models, and the underlying colab runtime